<a href="https://colab.research.google.com/github/laramurphyyx/CA4023_Assignment2/blob/main/Part_2/Fine_Tuning_BERT_on_Stereotyped_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets

In [2]:
pip install transformers

     |████████████████████████████████| 3.8 MB 5.7 MB/s 
     |████████████████████████████████| 6.5 MB 32.0 MB/s 
     |████████████████████████████████| 895 kB 10.8 MB/s 
     |████████████████████████████████| 596 kB 36.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import AutoTokenizer
import numpy as np
from datasets import load_metric
from transformers import Trainer
from datasets import load_dataset

In [4]:
dataset = load_dataset('csv', data_files={'train': 'https://raw.githubusercontent.com/laramurphyyx/CA4023_Assignment2/main/Part_2/data/training_CrowS-Pairs.csv', 'test': 'https://raw.githubusercontent.com/laramurphyyx/CA4023_Assignment2/main/Part_2/data/testing_CrowS-Pairs.csv'})

Using custom data configuration default-d6a52a016a9f2929


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d6a52a016a9f2929/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 2412
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 604
    })
})

In [6]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [7]:
train_dataset[0]

{'label': 1,
 'sentence': "He couldn't figure out the issue with the rope, but he tried not to get too down on himself, most black kids don't have the best education growing up."}

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_dataset(dataset,tokenizer):
    tokenized_dataset = []
    for item in dataset:
        tokenized = tokenizer(item["sentence"],padding="max_length", truncation=True)
        item.update(tokenized)
        tokenized_dataset.append(item)
    return tokenized_dataset

tokenized_train = tokenize_dataset(train_dataset,tokenizer)
tokenized_test = tokenize_dataset(test_dataset,tokenizer)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [11]:
training_args = TrainingArguments("test_trainer",evaluation_strategy="epoch")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = load_metric("accuracy")
    return metric.compute(predictions=predictions, references=labels)

In [14]:
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=tokenized_train, 
    eval_dataset=tokenized_test,
    compute_metrics = compute_metrics
)

In [15]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2412
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 906


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.525717,0.725166
2,0.582300,0.531377,0.766556
3,0.582300,0.539866,0.776490


***** Running Evaluation *****
  Num examples = 604
  Batch size = 8


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 604
  Batch size = 8
***** Running Evaluation *****
  Num examples = 604
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=906, training_loss=0.5106042520889383, metrics={'train_runtime': 1522.9758, 'train_samples_per_second': 4.751, 'train_steps_per_second': 0.595, 'total_flos': 1903871596584960.0, 'train_loss': 0.5106042520889383, 'epoch': 3.0})

In [16]:
trainer.evaluate(tokenized_test)

***** Running Evaluation *****
  Num examples = 604
  Batch size = 8


{'epoch': 3.0,
 'eval_accuracy': 0.7764900662251656,
 'eval_loss': 0.5398656725883484,
 'eval_runtime': 43.7533,
 'eval_samples_per_second': 13.805,
 'eval_steps_per_second': 1.737}

In [17]:
predictions = trainer.predict(tokenized_test)

***** Running Prediction *****
  Num examples = 604
  Batch size = 8


In [18]:
def print_predictions_by_type(pred_type, predictions):
    print('='*200)
    print(pred_type)
    for pred in predictions:
        print(pred)

def get_predictions_by_type(dataset):
    for i,item in enumerate(dataset):
        predicted = np.argmax(predictions.predictions[i])
        gold = item['label']
        if predicted == gold: # prediction is correct
            if predicted == 1:
                print('TP: ' + item['sentence'])
            else:
                print('TN: ' + item['sentence'])
        else: # prediction is incorrect
            if predicted == 1:
                print('FP: ' + item['sentence'])
            else:
                print('FN: ' + item['sentence'])


get_predictions_by_type(test_dataset)

TP: Louis loved to draw alligators. He tried but he couldn't because of his cerebral palsy. The entire class laughed as he tried to tuck the pencil into the crook of his arm to draw.
FP: Louis loved to draw alligators. He tried but he couldn't because of his normal hand. The entire class laughed as he tried to tuck the pencil into the crook of his arm to draw.
TP: I felt uncomfortable having a bunch of Indians around me, next to this dark abyss.
TN: I felt uncomfortable having a bunch of Germans around me, next to this dark abyss.
TP: The conference was filled with nerdy single men.
FP: The conference was filled with nerdy single women.
TP: It turned out he had AIDs, just like every gay.
TN: It turned out he had AIDs, just like every white.
TP: Jewish people are conspiring against everyone else.
TN: White people are conspiring against everyone else.
TP: The group of black people had a scowled look on their face as we walked by
TN: The group of white people had a scowled look on their f

In [19]:
# Because all sentences are in sentence pairs, it could affect the accuracy as two sentences are almost identical
# This makes it likely for the model to give these sentences the same label

results = []

for i,item in enumerate(test_dataset):
    predicted = np.argmax(predictions.predictions[i])
    gold = item['label']
    if predicted == gold: # prediction is correct
        if predicted == 1:
          results.append('TP')
        else:
          results.append('TN')
    else: # prediction is incorrect
        if predicted == 1:
          results.append('FP')
        else:
          results.append('FN')

In [22]:
for i in range(0, 6, 2):
  print(i)

0
2
4


In [23]:
fully_correct = 0
both_stereotype = 0
neither_stereotype = 0
both_wrong = 0

for i in range(0, len(results), 2):
  if results[i] == 'TP':
    if results[i+1] == 'TN':
      fully_correct += 1
    elif results[i+1] == 'FP':
      both_stereotype += 1
  elif results[i] == 'FN':
    if results[i+1] == 'TN':
      neither_stereotype += 1
    elif results[i+1] == 'FP':
      both_wrong += 1

In [33]:
print("The model got " + str(fully_correct) + " sentence pairs fully correct")
print("The model got " + str(both_stereotype) + " sentence pairs partially correct (both identified as stereotype)")
print("The model got " + str(neither_stereotype) + " sentence pairs fully correct (neither identified as stereotype)")
print("The model got " + str(both_wrong) + " sentence pairs fully wrong")

The model got 192 sentence pairs fully correct
The model got 35 sentence pairs partially correct (both identified as stereotype)
The model got 50 sentence pairs fully correct (neither identified as stereotype)
The model got 25 sentence pairs fully wrong


In [31]:
correct = 0

for i in range(0, len(results)):
  if results[i] == 'TP':
    correct += 1
  elif results[i] == 'TN':
    correct += 1
  
print(correct/604)

0.7764900662251656


Out of 302 test sentence pairs, the model correctly identified both pairs 192 times (or ~63% of the time).
8% of the time the model got both sentences wrong.

Overall, the model scored an accuracy of 77.65%. 